# Datasets overview (data mining)
Hereby a preliminary analysis of the dataset is performed to identify the available data.

### Active Plane Of Array (PoA) thermoelectric measurement datasets
To fulfill research objectives, available PoA thermoelectric pyranometers are first identified.

#### Extracting metadata infomation
First metadata information about PoA thermopile sensors (not on a tracker) are extracted.

In [13]:
#setup module always to be started
import pandas as pd
# for datetime stripping  https://www.tutorialspoint.com/python/time_strptime.htm
from datetime import datetime
#import seaborn for 'vlag' cmap
import seaborn as sn
import database_operations as db
import indoor_calibration_data as clb

def format_timestamp(s: str) -> datetime:
    f = '%Y-%m-%d %H:%M:%S.%f'
    return datetime.strptime(s.replace("T"," ")[:-3], f)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

In [2]:
engine = db.DatabaseEngine()
poa_sensors_info = engine.get_sensors_info()
poa_sensors_info

Query started at Wed Dec 16 19:09:28 2020
Query ended at Wed Dec 16 19:09:29 2020


,sensorname,sensorcomment,manufacturer,manufacturermodelnumber,manufacturerserialnumber,lastcalibrationdate,systemname,nameoflocation,latitude,longitude,altitude,systemazimuth,systeminclination,systemcomment
0,Pyro_CM22_W01,Was ID 3,Kipp & Zonen,CM22,040099,2014-11-03,COMS-3.1: meteorological subsystem,CREST W roof northerly-high upstand South-facing 45deg,52.761582,-1.240617,79.0,-0.73,45.6,Direct measure system based on NI Mseries card and signal conditioning and commercial multiplexers.
1,Pyro_CMP11_W03,None,Kipp & Zonen,CMP11,060118,None,Ketut Longterm,CREST Wroof north end-low upstand South 45deg,52.761556,-1.240707,79.0,-0.59,44.7,This is Ketut Astawas main data acquisition system for his PhD and takes similar data to COMS3
2,Pyro_CM11_W01,Indoor cal,Kipp & Zonen,CM11,027700,2014-10-31,COMS-3.1: meteorological subsystem,CREST W roof northerly-high upstand South-facing 45deg,52.761582,-1.240617,79.0,-0.73,45.6,Direct measure system based on NI Mseries card and signal conditioning and commercial multiplexers.
3,pyro_cmp11_w05,MMS1 on highest rack near lift indoor cal,Kipp&Zonen,CMP11,80286,2014-11-03,MMS1 Met Monitoring System NE end fixed sensors,CREST W roof northerly-high upstand South-facing 45deg,52.761582,-1.240617,79.0,-0.73,45.6,Built by Michal for Ikea project. uses CR1000 and Mux to measure multitude of env sensors and full size modules as reference cells. data initially in seperate MySQL DB
4,pyro_cmp11_w06,indoor cal,Kipp&Zonen,CMP11,115538,2014-10-31,MMS1 Met Monitoring System NE end fixed sensors,CREST Wroof NE end-low upstand South 34 deg,52.761556,-1.240707,79.0,-0.59,32.5,Built by Michal for Ikea project. uses CR1000 and Mux to measure multitude of env sensors and full size modules as reference cells. data initially in seperate MySQL DB
5,Pyro_CMP11_W09,"in CVF4 vent unit, on GRP grid, on southernmost upstand rack",Kipp&Zonen,CMP11,140092,2014-01-21,MMS3 Met monitoring system on SE upstand rack,CREST Wroof south-east-area upstand South 34deg,52.761411,-1.240531,79.0,-0.17,33.9,"Similar to MMS1 & MMS2, but has 1 CR1000 & 2 CR800s"


#### Finding relevant monitored PoA parameters
The found sensor names are used to identify where useful monitoring data are located.
First numerical identifiers are extracted.
Then acceptable and unacceptable strings for columns, tables and schemas are defined.

In [2]:
poa_columns_information = engine.get_columns_information_schema()
poa_columns_information

,table_schema,table_name,column_name,data_type
0,w_meas,mms1v1met,pyro_cmp11_w06_wm2,numeric
1,w_meas,coms3_met,cm11testi,numeric
2,w_meas,coms3_met,pyro_cmp11_w08,numeric
3,w_meas,mms1v1met,pyro_cmp11_w04,numeric
4,w_meas,mms1v1met,pyro_cmp11_w05,numeric
5,w_meas,mms1v1met,pyro_cmp11_w06,numeric
6,w_meas,coms3_met,pyro_cm11_w01,numeric
7,w_meas,coms3_met,pyro_cmp11_w01,numeric
8,w_meas,coms3_met,pyro_cmp11_w02,numeric
9,w_meas,coms3_met,pyro_cm22_w02,numeric


#### Extracting aggregate functions to analyse PoA parameters
General purpose (faster) aggregate functions (count, min, max, avg and sum) are extracted from the dataset,
as well as the first and last timestamp for the which the measured parameter was not null.

In [ ]:
poa_columns_description = engine.get_columns_description(poa_columns_information)
poa_columns_description

Some columns have only null values and were excluded further analysis

In [4]:
engine.columns_dropped

,schemas,table,column,cause
0,w_meas,mms1v1met,pyro_cmp11_w06_wm2,only null values
1,w_meas,coms3_met,cm11testi,only null values


#### Analysis of found PoA parameters
Overview table was previously saved as *columns_description.csv* due to the long processing time.
The file is loaded again preserving only the useful information.
All the found valid PoA parameters are located in *w_meas* database and are of *numeric* type.

In [12]:
# https://www.tutorialspoint.com/python/time_strptime.htm

poa_columns_description  = pd.read_csv("poa_columns_description.csv",
                                   infer_datetime_format = True,
                                   usecols = ['column_name', 'timestamp_column', 'timestamp_min',
                                              'timestamp_max',	'count', 'min', 'max', 'sum', 'avg'],
                                   converters= {'timestamp_min': lambda x: format_timestamp(x),
                                              'timestamp_max': lambda x: format_timestamp(x)})
#avoid showing table now since formatted later
print("")

,column_name,timestamp_column,timestamp_min,timestamp_max,count,min,max,sum,avg
0,pyro_cmp11_w08,measurementdatetime,2015-03-31 09:20:55,2015-04-07 10:10:15,121495.0,-25.392,422.357,9.007372e+06,74.137798
1,pyro_cmp11_w04,measurementdatetime,2009-11-26 02:01:22,2016-04-11 12:34:59,150397930.0,-9999.000,58.750,-2.336121e+08,-1.553293
2,pyro_cmp11_w05,measurementdatetime,2009-11-26 02:01:22,2016-04-11 12:34:59,150397930.0,-9999.000,120.000,-6.643747e+07,-0.441745
3,pyro_cmp11_w06,measurementdatetime,2013-04-24 00:00:00,2016-04-11 12:34:59,58364327.0,-365.400,22.340,5.999954e+07,1.028017
4,pyro_cm11_w01,measurementdatetime,2006-10-16 10:20:25,2016-01-03 23:59:55,49011794.0,-3333.000,3333.000,7.034095e+09,143.518415
5,pyro_cmp11_w01,measurementdatetime,2007-02-26 09:21:55,2014-11-05 12:09:55,510849.0,-3333.000,3333.000,6.806141e+07,133.231957
6,pyro_cmp11_w02,measurementdatetime,2007-03-23 09:42:35,2016-01-03 23:59:55,6384249.0,-3333.000,3333.000,7.468282e+09,1169.797998
7,pyro_cm22_w02,measurementdatetime,2006-10-30 12:43:35,2016-01-03 23:59:55,38608677.0,-3333.000,3333.000,4.388886e+09,113.676163
8,pyro_cm22_w01,measurementdatetime,2006-10-16 10:20:25,2016-01-03 23:59:55,56196938.0,-3333.000,3333.000,7.863325e+09,139.924439
9,pyro_cmp11_w07,measurementdatetime,2015-01-16 15:56:20,2016-04-11 12:40:02,35537473.0,-6.345,1385.000,3.502405e+09,98.555257


Average resolution is calculate for each parameter and table is formatted with a colour gradient for an easier analysis.

In [21]:
%matplotlib inline
poa_columns_description["resolution_estimated"] = ((poa_columns_description.timestamp_max - poa_columns_description.timestamp_min)/
                                            poa_columns_description["count"])
poa_columns_description.resolution_estimated = poa_columns_description.resolution_estimated.transform(lambda x: x.total_seconds())

poa_columns_description.style.background_gradient(cmap='vlag')
#additional information on colour formatting at:
#https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
#https://seaborn.pydata.org/tutorial/color_palettes.html

,column_name,timestamp_column,timestamp_min,timestamp_max,count,min,max,sum,avg,resolution_estimated
0,pyro_cmp11_w08,measurementdatetime,2015-03-31 09:20:55,2015-04-07 10:10:15,121495.000000,-25.392000,422.357000,9007371.793000,74.137798,5.002345
1,pyro_cmp11_w04,measurementdatetime,2009-11-26 02:01:22,2016-04-11 12:34:59,150397930.000000,-9999.000000,58.750000,-233612098.397000,-1.553293,1.337632
2,pyro_cmp11_w05,measurementdatetime,2009-11-26 02:01:22,2016-04-11 12:34:59,150397930.000000,-9999.000000,120.000000,-66437471.916000,-0.441745,1.337632
3,pyro_cmp11_w06,measurementdatetime,2013-04-24 00:00:00,2016-04-11 12:34:59,58364327.000000,-365.400000,22.340000,59999539.868000,1.028017,1.604002
4,pyro_cm11_w01,measurementdatetime,2006-10-16 10:20:25,2016-01-03 23:59:55,49011794.000000,-3333.000000,3333.000000,7034094985.207000,143.518415,5.934726
5,pyro_cmp11_w01,measurementdatetime,2007-02-26 09:21:55,2014-11-05 12:09:55,510849.000000,-3333.000000,3333.000000,68061412.225000,133.231957,475.106499
6,pyro_cmp11_w02,measurementdatetime,2007-03-23 09:42:35,2016-01-03 23:59:55,6384249.000000,-3333.000000,3333.000000,7468281700.412000,1169.797998,43.422905
7,pyro_cm22_w02,measurementdatetime,2006-10-30 12:43:35,2016-01-03 23:59:55,38608677.000000,-3333.000000,3333.000000,4388886255.452000,113.676163,7.502287
8,pyro_cm22_w01,measurementdatetime,2006-10-16 10:20:25,2016-01-03 23:59:55,56196938.000000,-3333.000000,3333.000000,7863325025.480000,139.924439,5.175932
9,pyro_cmp11_w07,measurementdatetime,2015-01-16 15:56:20,2016-04-11 12:40:02,35537473.000000,-6.345000,1385.000000,3502404799.888000,98.555257,1.096156


#### Conclusions on PoA dataset
Only *g_cmp11_ppuk* is selected for future analysis due the following reasons:
- Monitoring up to the present time (since 2013).
- High resolution data (1s)
- Highest number of valid data (about 200 millions)
- Realistic minimum (-6.3 W/m2) and average (141.12 W/m2)

Maximum value of 1611 W/m2 seems also quite realistic since the other realistic values correspond to shorter timeframes.

Estimated resolution of 1.19 shows the presence of unwanted null values which should be addressed before further analysis.

No information are available on where *g_cmp11_ppuk* is located but '11' sensors are in four locations:
Overview table of PoA sensors

In [8]:
poa_sensors = poa_sensors_info.loc[poa_sensors_info.sensorname.str.contains('11'),
                                   ["sensorname","manufacturerserialnumber","sensorcomment", "lastcalibrationdate", "nameoflocation"]]
poa_sensors

,sensorname,manufacturerserialnumber,sensorcomment,lastcalibrationdate,nameoflocation
1,Pyro_CMP11_W03,060118,None,None,CREST Wroof north end-low upstand South 45deg
2,Pyro_CM11_W01,027700,Indoor cal,2014-10-31,CREST W roof northerly-high upstand South-facing 45deg
3,pyro_cmp11_w05,80286,MMS1 on highest rack near lift indoor cal,2014-11-03,CREST W roof northerly-high upstand South-facing 45deg
4,pyro_cmp11_w06,115538,indoor cal,2014-10-31,CREST Wroof NE end-low upstand South 34 deg
5,Pyro_CMP11_W09,140092,"in CVF4 vent unit, on GRP grid, on southernmost upstand rack",2014-01-21,CREST Wroof south-east-area upstand South 34deg


Unique locations found:

In [12]:
[print(location) for location in poa_sensors["nameoflocation"].unique().tolist()]

CREST Wroof north end-low upstand South 45deg
CREST W roof northerly-high upstand South-facing 45deg
CREST Wroof NE end-low upstand South 34 deg
CREST Wroof south-east-area upstand South 34deg


[None, None, None, None]

For *pyro_cmp11_w06* (id 115538) KZ calibration certificate 005897115538 was found (21 November 2011).
No information about the abovementioned id is found among the indoor calibration files (starting from 29/07/2015).
Among the PoA sensor only one calibration (file dated 20/07/20) is found and it is related to serial number 060118.

**UPDATE 2/12/20 (email)**
It is confirmed that *g_cmp11_ppuk* refers to *pyro_cmp11_W06* and that the sensor is not ventilated
(ventilation was not mentioned in the description). Original found calibration date of 21/11/20 is also confirmed.
Sensor was recalibrated indoor on 31/10/2014 (confirmed sensitivity of 8.80 uV/Wm-2)
but no calibration file is available in the availabe series from 2015.

In [20]:
calibrations_info = clb._main_info_to_dataframe()
search_str = "|".join(poa_sensors.manufacturerserialnumber.to_list())
calibration_info_poa_sensors = pd.DataFrame(columns=calibrations_info.columns)
columns_to_be_explored = ['filename', 'sample_id', 'sample_type', 'sample_details']
#aggregate detailed information into one field for better visualisation
calibrations_info['sample_details'] = calibrations_info['sample_details'].transform(lambda x: " ".join([x for x in x]))
for i, row in calibrations_info[columns_to_be_explored].iterrows():
    #look for found manufacturer serial numbers in the dataset
    _ = row.str.contains(search_str)
    if any(_):
        calibration_info_poa_sensors = calibration_info_poa_sensors.append(calibrations_info.loc[i,:], ignore_index=True)
calibration_info_poa_sensors

,filename,sample_id,sample_type,sample_details,reference_id,reference_type,reference_sensitivity [uV/Wm-2],wacom_id,wacom_type,meas_temp,meas_time,response_av [uV/Wm-2],response_std [uV/Wm-2],irradiance_av [uV/Wm-2],irradiance_std [uV/Wm-2]
0,WAC-010-20180720123750-CMP11060118-000-RESTRICTED.txt,CMP11060118,CMP11,"""""",CMP21-140465,K&Z,8.610000000E+0,RCCWAR-Q087,c-Si,25.02,2018-07-20 09:32:14,8.705891315E+0,4.548413755E-3,1.246032938E+3,2.298632090E-1


### Modules measurement datasets
To fulfill research objectives, modules monitored along *g_cmp11_ppuk* are first identified based on the found locations.

#### Extracting aggregate functions to analyse PoA parameters
First isc columns are described similary to what done for the poa irradiance parameters.

In [5]:
isc_columns_information = engine.get_columns_information_schema(column_numbers_included = [''],
    column_strings_included = ["isc"])
isc_columns_information

,table_schema,table_name,column_name,data_type,timestamp_column
0,w_meas,mms1v1met,isc_refmod_sharp,numeric,measurementdatetime
1,w_meas,mms1v1met,isc_refmod_sunpower,numeric,measurementdatetime
2,w_meas,mms1v1met,isc_refmod_rec,numeric,measurementdatetime
3,w_meas,mms1v1met,isc_refmod_yingli,numeric,measurementdatetime
4,w_meas,mms1v1met,isc_refmod_wurth,numeric,measurementdatetime
5,w_meas,pv_ivt_tom,isc,numeric,obstimestamp


1h of processing is required thus information is stored into file "isc_columns_description.csv".

In [ ]:
text_stream =  open("isc_columns_description.csv", 'w')

clm_inf = engine.get_columns_description(
    column_numbers_included = [''],
    column_strings_included = ["isc"],
    text_stream= text_stream)

In [2]:
isc_columns_description  = pd.read_csv("isc_columns_description.csv",
                                   infer_datetime_format = True,
                                   usecols = ['column_name', 'timestamp_column', 'timestamp_min',
                                              'timestamp_max',	'count', 'min', 'max', 'sum', 'avg'],
                                   converters= {'timestamp_min': lambda x: format_timestamp(x),
                                              'timestamp_max': lambda x: format_timestamp(x)})

%matplotlib inline
isc_columns_description["resolution_estimated"] = ((isc_columns_description.timestamp_max - isc_columns_description.timestamp_min)/
                                            isc_columns_description["count"])
isc_columns_description.resolution_estimated = isc_columns_description.resolution_estimated.transform(lambda x: x.total_seconds())

isc_columns_description.style.background_gradient(cmap='vlag')

,column_name,timestamp_column,timestamp_min,timestamp_max,count,min,max,sum,avg,resolution_estimated
0,isc_refmod_sharp,measurementdatetime,2009-11-26 02:01:22,2014-12-31 00:00:00,116432681.000000,-9999.000000,14.190000,-103354741982.028015,-887.678108,1.380910
1,isc_refmod_sunpower,measurementdatetime,2009-11-26 02:01:22,2014-12-31 00:00:00,116432681.000000,-9999.000000,9.993000,-103308416975.416992,-887.280239,1.380910
2,isc_refmod_rec,measurementdatetime,2009-11-26 02:01:22,2014-12-31 00:00:00,116432681.000000,-9999.000000,14.327000,-103264179877.225998,-886.900301,1.380910
3,isc_refmod_yingli,measurementdatetime,2009-11-26 02:01:22,2014-12-31 00:00:00,116432681.000000,-9999.000000,23.230000,-103328010308.554001,-887.448519,1.380910
4,isc_refmod_wurth,measurementdatetime,2009-11-26 02:01:22,2014-12-11 14:17:15,115596099.000000,-9999.000000,4.012000,-103362268323.975998,-894.167444,1.376400
5,isc,numeric,2013-03-04 15:56:00,2020-06-30 23:59:02,32383354.000000,-0.018000,12.174000,31750457.859000,0.980456,7.137895


#### Conclusions on isc dataset
Only *isc* is selected for future analysis due the following reasons:
- Monitoring up to the present time (since 2013).
- Realistic minimum (-0.018000) and average (0.980456) value.
- Highest number of valid data (about 32 millions)

Maximum value of 12.17 far from 23.23 (isc_refmod_yingli) but still in line with other isc parameters.
Estimated resolution of 7.14 to be explored further for each module inside the dataset.

### pv_ivt_tom dataset
Before retrieving modules data, isc pv_ivt_tom table  needs to be explored further.
Null information are removed and information shared by all the parameters are stored separately.

In [88]:
pv_ivt_info_per_column, pv_ivt_info_all_columns = engine.get_table_information()
print(pv_ivt_info_all_columns)
pv_ivt_info_per_column

{'table_catalog': 'APVDBV3', 'table_schema': 'w_meas', 'table_name': 'pv_ivt_tom', 'udt_catalog': 'APVDBV3', 'udt_schema': 'pg_catalog', 'is_self_referencing': 'NO', 'is_identity': 'NO', 'is_generated': 'NEVER', 'is_updatable': 'YES'}



,column_name,ordinal_position,column_default,is_nullable,data_type,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,udt_name,dtd_identifier,description
0,channelnumber,1,None,NO,smallint,16.0,2.0,0.0,NaN,int2,1,Channels in the 800s range are Egnitec COMS4 h...
1,obstimestamp,2,None,NO,timestamp without time zone,NaN,NaN,NaN,6.0,timestamp,2,None
2,impp,3,None,YES,numeric,5.0,10.0,3.0,NaN,numeric,3,None
3,vmpp,4,None,YES,numeric,6.0,10.0,3.0,NaN,numeric,4,None
4,isc,5,None,YES,numeric,5.0,10.0,3.0,NaN,numeric,5,None
5,voc,6,None,YES,numeric,6.0,10.0,3.0,NaN,numeric,6,None
6,moduletemperature1,7,None,YES,numeric,6.0,10.0,3.0,NaN,numeric,7,None
7,moduletemperature2,8,None,YES,numeric,6.0,10.0,3.0,NaN,numeric,8,None
8,iv_sweep_endtime,9,None,NO,timestamp without time zone,NaN,NaN,NaN,6.0,timestamp,9,None
9,no_of_points,10,None,YES,smallint,16.0,2.0,0.0,NaN,int2,10,None


Assuming the sweep time very short comparing to the interval between each sweep,
*iv_sweep_endtime* can be identified as reference timestamp for isc.
Only rows having a value of 1 should be considered for *errorflag* following the column description.

Closing connection to the database at the end.

In [14]:
#ending connection always to be started
engine.connection.close()